merge long and wide format of unemployment rate data to time-series data

In [1]:
import pandas as pd
import datetime
import re

In [2]:
df_wide = pd.read_csv('alq_wide.csv', index_col=0)
df_long = pd.read_csv('alq_long.csv', index_col=0)

In [3]:
df_wide.shape

(401, 169)

In [4]:
df_long.shape

(67769, 3)

In [5]:
df_long.dtypes

ags5            int64
time_stamp     object
value         float64
dtype: object

## merge

In [6]:
df_final_wide = pd.read_csv('../../final_dfs/for_modeling/df_final_date_wide_2007.csv', index_col=0)
df_final_long = pd.read_csv('../../final_dfs/for_modeling/df_final_date_long_2007.csv', index_col=0)

In [7]:
df_final_wide.shape

(67368, 40)

In [8]:
df_final_long.shape

(599980, 5)

In [9]:
# fix variable name
df_final_wide.rename(
    columns={"employees_social_security_at_residenceemployees_social_security_at_work": 
             "employees_social_security_at_work"}, inplace=True)

In [10]:
def fix_variable(var):
    if var=='employees_social_security_at_residenceemployees_social_security_at_work':
        var = 'employees_social_security_at_work'
        return var
    else:
        return var

In [11]:
df_final_long['variable'] = df_final_long['variable'].apply(fix_variable)

In [12]:
# fix date to first day of the month to merge
def fix_day(date):
    return date.replace(day=1)

### wide

In [13]:
df_final_wide.shape

(67368, 40)

In [14]:
df_wide.shape

(401, 169)

In [15]:
#df_final_wide

In [16]:
df_long['datetime'] = pd.to_datetime(df_long['time_stamp'], format='%Y-%m-%d')

In [17]:
# filter these out to avoid duplicate when merging
#df_long = df_long[df_long['datetime']<datetime.datetime(2019,1,1)]

In [18]:
df_long['date'] = df_long['datetime'].apply(fix_day)

In [19]:
df_long = df_long[['ags5', 'date', 'value']]

In [20]:
df_long.columns = ['ags5', 'date', 'unemployment_rate']

In [21]:
df_final_wide['date'] = pd.to_datetime(df_final_wide['date'])

In [22]:
# drop alq from original data and update directly from new data
df_final_wide = df_final_wide.drop(columns=['unemployment_rate'])

In [23]:
df_final_wide = pd.merge(df_final_wide, df_long, left_on=['ags5', 'date'], right_on=['ags5', 'date'])

In [24]:
# df_final_wide

### long

In [25]:
df_final_long.shape

(599980, 5)

In [26]:
df_long.shape

(67769, 3)

In [27]:
df_final_long['date'] = pd.to_datetime(df_final_long['date'])

In [28]:
# drop alq from original data
df_final_long = df_final_long[df_final_long['variable']!='unemployment_rate']

In [29]:
# # remove '-' and add 'd' for wide_to_long conversions
# df_wide = df_wide.rename(columns=lambda x: re.sub('-', '', x))
# df_wide = df_wide.rename(columns=lambda x: re.sub(r'([^\s]+)', 'd\\1', x))

In [30]:
# df_wide.reset_index(inplace=True)
# df_wide.rename(columns={"index": "ags5"}, inplace=True)

In [31]:
#df_wide

In [32]:
# df_wide = pd.wide_to_long(df_wide, i=['ags5'], stubnames='d', j='date').reset_index()

In [33]:
# df_wide.rename(columns={"d": "ags5"}, inplace=True)

In [34]:
#df_final_long

In [35]:
df_long['ags2'] = df_long['ags5'].astype(str).str[:-3]
df_long.rename(columns={"unemployment_rate": "value"}, inplace=True)
df_long['variable'] = 'unemployment_rate'

In [36]:
df_long = df_long[['ags2', 'ags5', 'variable', 'date', 'value']]

In [37]:
df_final_long = pd.concat([df_final_long, df_long], ignore_index=True)

In [38]:
# df_final_wide.to_csv('../../final_dfs/for_modeling/df_final_date_wide_2007.csv')
# df_final_long.to_csv('../../final_dfs/for_modeling/df_final_date_long_2007.csv')